# Inventory Control with Lead Times and Multiple Suppliers

## Description
One potential application of reinforcement learning involves ordering supplies from multiple suppliers with varying lead times and costs to meet a changing demand.

The inventory control problem addresses the problem by modeling an environment where there are multiple suppliers with different costs and lead times. Orders must be placed with these suppliers to have an on-hand inventory to meet a changing demand. However, both having supplies on backorder and holding unused inventory have associated costs. The goal of the agent is to choose the amount to order from each supplier to maximize the revenue earned. 

At each time step, an order is placed to each supplier. If previous orders have waited for the length of their supplier's lead time, than these orders will become part of the on-hand inventory. The demand is then randomly chosen from a user-selected distribution and is subtracted from the on-hand inventory. If the on-hand inventory would become less than zero, than items are considered to be on backorder which decreases the reward. However, the on-hand inventory will be zero for the next time step. Otherwise, the demand is subtracted from the on-hand inventory to calculate on-hand inventory for the start of the next time step. If there is remaining inventory (a nonzero number) at the end of this calculation, this number is being held, and negatively influences the reward proportional to the holding costs. 

## Model Assumptions
* Inventory can not be negative. If the inventory is to go negative, it becomes zero.
* Backorders are not retroactively fulfilled. If a high demand would cause inventory to become negative, this unfulfilled demand is not met later when there may be some inventory being held at the end of a timestep.

## Dynamics
### State Space
The state space is $S = Z^{L_1} * Z^{L_2} * ... * Z^{L_N} * I$ where $N$ is the number of suppliers and $Z^{L_i}$ represents a list of nonnegative integers with the length of the lead time of supplier $i$. $I$ represents the current on-hand inventory. This represents how many timesteps back each order is from being added to the inventory. To represent a timestep, an order will be moved up an index in the array unless it is added to the inventory, in which case it is removed from the array. Each supplier has their own set of indices in the array that represent its lead times.
### Action Space
The action space is $A = Z^N$ where N is the number of suppliers. This represents the amount to order from each supplier for the current timestep.
### Reward

### Transitions


## Environment

## Heuristic Agents